<a href="https://colab.research.google.com/github/Dimavl2025/skin_cancer_clasification/blob/main/dinoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<big>First setp is to train the dinov2 classifier with labeled data</big>

*Pre requirement:*

*pip install torch torchvision timm scikit-learn*


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ============================================
# 0. Install deps (run once per new runtime)
# ============================================
!pip install -q transformers scikit-learn

# ============================================
# 1. Imports & setup
# ============================================
import os
import random
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import Dinov2Config, Dinov2ForImageClassification, AutoImageProcessor


# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================
# 2. Paths & experiment definitions
# ============================================
MAIN_DIR = "/content/drive/MyDrive/Colab Notebooks/Course/Skin_Cancer_Classification"

EXPERIMENTS = [
    {
        "pos_folder": "1_generated_from_augmentation",
        "exp_name": "dinov2_1_generated_from_augmentation"
    },
    {
        "pos_folder": "1_generated from unlabeled",  # note the space
        "exp_name": "dinov2_1_generated_from_unlabeled"
    }
]

SAVE_MODELS_DIR = os.path.join(MAIN_DIR, "models")
os.makedirs(SAVE_MODELS_DIR, exist_ok=True)

# ============================================
# 3. Image processor for DINOv2
#    (only config & preprocessing, NO weights)
# ============================================
# We re-use the official DINOv2 image processor to get
# correct resize/normalization, but the model itself will
# be created from a config (random weights).
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-small")

# ============================================
# 4. Dataset definition
# ============================================
IMG_EXTENSIONS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp")

class SkinCancerDataset(Dataset):
    def __init__(self, image_paths, labels, processor):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")
        inputs = self.processor(images=image, return_tensors="pt")
        # pixel_values shape: [1, 3, H, W] -> [3, H, W]
        pixel_values = inputs["pixel_values"].squeeze(0)

        return {
            "pixel_values": pixel_values,
            "labels": torch.tensor(label, dtype=torch.long)
        }

def collect_image_paths(folder):
    paths = []
    for root, _, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(IMG_EXTENSIONS):
                paths.append(os.path.join(root, f))
    return paths

# ============================================
# 5. Training & evaluation helpers
# ============================================
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for batch in tqdm(dataloader, desc="Train", leave=False):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits  # [B, 2]

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples
    return avg_loss, acc

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Val", leave=False):
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(pixel_values=pixel_values)
            logits = outputs.logits

            loss = criterion(logits, labels)
            total_loss += loss.item() * labels.size(0)

            preds = torch.argmax(logits, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy().tolist())
            all_preds.extend(preds.cpu().numpy().tolist())

    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples

    # Binary metrics for positive class = 1
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="binary", pos_label=1, zero_division=0
    )

    return avg_loss, acc, precision, recall, f1

def plot_curves(train_losses, val_losses, train_accs, val_accs, exp_name):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Train Loss")
    plt.plot(epochs, val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Loss - {exp_name}")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accs, label="Train Acc")
    plt.plot(epochs, val_accs, label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy - {exp_name}")
    plt.legend()

    plt.tight_layout()
    plt.show()

# ============================================
# 6. Main training loop for each experiment
# ============================================
NUM_EPOCHS = 30
BATCH_SIZE = 16   # adjust if you see OOM
LR = 1e-4
VAL_SPLIT = 0.2   # 80% train / 20% val

for cfg in EXPERIMENTS:
    pos_folder = cfg["pos_folder"]
    exp_name = cfg["exp_name"]

    print("\n" + "="*80)
    print(f"Starting experiment: {exp_name}")
    print(f"Positive folder: {pos_folder}")
    print("="*80)

    # 1) Collect image paths
    neg_dir = os.path.join(MAIN_DIR, "0")
    pos_dir = os.path.join(MAIN_DIR, pos_folder)

    neg_images = collect_image_paths(neg_dir)
    pos_images = collect_image_paths(pos_dir)

    print(f"Found {len(neg_images)} negative images (class 0)")
    print(f"Found {len(pos_images)} positive images (class 1)")

    all_paths = neg_images + pos_images
    all_labels = [0] * len(neg_images) + [1] * len(pos_images)

    # 2) Train/val split (stratified)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        all_paths,
        all_labels,
        test_size=VAL_SPLIT,
        random_state=SEED,
        stratify=all_labels,
    )

    print(f"Train size: {len(train_paths)}, Val size: {len(val_paths)}")

    # 3) Build datasets & loaders
    train_dataset = SkinCancerDataset(train_paths, train_labels, processor)
    val_dataset = SkinCancerDataset(val_paths, val_labels, processor)

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    # 4) Initialize DINOv2 model from CONFIG (NO pretrained weights)
    config = Dinov2Config.from_pretrained(
    "facebook/dinov2-small",
    num_labels=2
)
    # we still initialize RANDOM weights (no pretrained checkpoint)
    model = Dinov2ForImageClassification(config)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR)

    # 5) Training loop
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")

        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, criterion)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(
            f"Train  - loss: {train_loss:.4f}, acc: {train_acc:.4f}\n"
            f"Val    - loss: {val_loss:.4f}, acc: {val_acc:.4f}, "
            f"P1: {val_prec:.4f}, R1: {val_rec:.4f}, F1: {val_f1:.4f}"
        )

    # 6) Final evaluation on val set
    val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, criterion)
    print("\n" + "-"*80)
    print(f"Final validation metrics for {exp_name}:")
    print(f"Loss: {val_loss:.4f}")
    print(f"Accuracy: {val_acc:.4f}")
    print(f"P1 (Precision for class 1): {val_prec:.4f}")
    print(f"R1 (Recall for class 1):    {val_rec:.4f}")
    print(f"F1 (F1 for class 1):        {val_f1:.4f}")
    print("-"*80)

    # 7) Plot curves
    plot_curves(train_losses, val_losses, train_accs, val_accs, exp_name)

    # 8) Save model & processor for later predictions
    save_dir = os.path.join(SAVE_MODELS_DIR, exp_name)
    os.makedirs(save_dir, exist_ok=True)
    model.save_pretrained(save_dir)
    processor.save_pretrained(save_dir)

    print(f"Saved model + processor to: {save_dir}")

print("\nAll experiments finished.")
